# 美联储FOMC官员讲话数据EDA ❤

@AUTHOR: WBZHANG / 01208663

@DATE: Oct. 31, 2024

In [ ]:
import os
import sys
sys.path.append("../")
import json
import glob
import numpy as np
import pandas as pd
from utils.file_saver import json_load
from utils.common import parse_datestring

FED_SPEECHES_PATH = "../data/fed_speeches/"

speech_dirs = glob.glob(FED_SPEECHES_PATH + "*_speeches")
speech_dirs = [x.replace("\\", "/") for x in speech_dirs]
print(speech_dirs)

['../data/fed_speeches/atlanta_fed_speeches', '../data/fed_speeches/bog_fed_speeches', '../data/fed_speeches/boston_fed_speeches', '../data/fed_speeches/chicago_fed_speeches', '../data/fed_speeches/cleveland_fed_speeches', '../data/fed_speeches/dallas_fed_speeches', '../data/fed_speeches/kansascity_fed_speeches', '../data/fed_speeches/minneapolis_fed_speeches', '../data/fed_speeches/newyork_fed_speeches', '../data/fed_speeches/philadelphia_fed_speeches', '../data/fed_speeches/richmond_fed_speeches', '../data/fed_speeches/sanfrancisco_fed_speeches', '../data/fed_speeches/stlouis_fed_speeches']


## 1. 加载数据集

In [33]:
CHOSEN_INDEX = 2
speeches_filapath = speech_dirs[2]
print("speeches filepath: {}".format(speeches_filapath))

speech_files = glob.glob(speeches_filapath + "//*_speeches.json")
print(speech_files)
# 加载数据
speeches_by_year = json_load(speech_files[0])

speeches filepath: ../data/fed_speeches\boston_fed_speeches
['../data/fed_speeches\\boston_fed_speeches\\boston_fed_speeches.json']


## 2. 探索性分析

In [34]:
# 年份跨度
start_year, end_year = min(speeches_by_year.keys()), max(speeches_by_year.keys())
print(
    f"{len(speeches_by_year)} years of {speech_files[0]} speeches was collected. \n StartYear: {start_year} - EndYear: {end_year} "
)
# 日期跨度
speech_dates = [
    parse_datestring(v["date"])
    for year, single_year_speeches in speeches_by_year.items()
    for v in single_year_speeches
]
begin_date = min(speech_dates)
end_date = max(speech_dates)
print(
    f"{len(speech_dates)} speeches was collected. \n BeginDate: {begin_date.date()} / EndDate: {end_date.date()} "
)

19 years of ../data/fed_speeches\boston_fed_speeches\boston_fed_speeches.json speeches was collected. 
 StartYear: 2006 - EndYear: 2024 
215 speeches was collected. 
 BeginDate: 2006-01-06 / EndDate: 2024-10-09 


In [35]:
# speech 正文长度统计
content_length = [
    len(v["content"])
    for year, single_year_speeches in speeches_by_year.items()
    for v in single_year_speeches
]
print(min(content_length), ' - ', max(content_length))

0  -  28701


In [ ]:
speeches = []
for date, single_year_records in speeches_by_year.items():
    single_year_speeches = pd.DataFrame.from_records(single_year_records)
    single_year_speeches["year"] = date
    speeches.append(single_year_speeches)
speeches = pd.concat(speeches, axis=0)
# 日期格式转换
speeches["date"] = pd.to_datetime(speeches["date"])
# 正文长度
speeches["content_length"] = speeches["content"].transform(lambda x: len(x))
speeches.sort_values(by="date", ascending=False, inplace=True)
print(speeches.info())
speeches.head()

<class 'pandas.core.frame.DataFrame'>
Index: 215 entries, 0 to 4
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   speech_title    215 non-null    object        
 1   highlights      215 non-null    object        
 2   content         215 non-null    object        
 3   date            215 non-null    datetime64[ns]
 4   title           215 non-null    object        
 5   href            215 non-null    object        
 6   summary         215 non-null    object        
 7   speaker         215 non-null    object        
 8   year            215 non-null    object        
 9   content_length  215 non-null    int64         
dtypes: datetime64[ns](1), int64(1), object(8)
memory usage: 18.5+ KB
None


,speech_title,highlights,content,date,title,href,summary,speaker,year,content_length
0,"Observations on the Economy, and the Vibrancy ...",Worcester Regional Research Bureau’s 39th Annu...,Takeaways from Boston Fed President Susan M. C...,2024-10-09,"Observations on the Economy, and the Vibrancy ...",https://www.bostonfed.org/news-and-events/spee...,Worcester Regional Research Bureau’s 39th Annu...,"Susan M. Collins, President & Chief Executive ...",2024,3150
1,Remarks at the 23rd Annual Regional and Commun...,Federal Reserve Bank of Boston’s 22nd Annual R...,Boston Fed President & CEO Susan M. Collins of...,2024-10-08,Remarks at the 23rd Annual Regional and Commun...,https://www.bostonfed.org/news-and-events/spee...,Federal Reserve Bank of Boston’s 22nd Annual R...,"Susan M. Collins, President & Chief Executive ...",2024,143
2,Welcoming Remarks at the forum on “Meeting the...,,Collins offered brief welcome remarks at a com...,2024-09-30,Welcoming Remarks at the forum on “Meeting the...,https://www.bostonfed.org/news-and-events/spee...,,"Susan M. Collins, President & Chief Executive ...",2024,137
3,A Partnership for Progress,Remarks to members of the Merrimack Valley com...,Takeaways from Boston Fed President Susan M. C...,2024-06-18,A Partnership for Progress,https://www.bostonfed.org/news-and-events/spee...,Remarks to members of the Merrimack Valley com...,"Susan M. Collins, President & Chief Executive ...",2024,1834
4,Reflections on Uncertainty and Patience in Mon...,Remarks at the Sloan School of Management at t...,Boston Fed President and CEO Susan M. Collins'...,2024-05-08,Reflections on Uncertainty and Patience in Mon...,https://www.bostonfed.org/news-and-events/spee...,Remarks at the Sloan School of Management at t...,"Susan M. Collins, President & Chief Executive ...",2024,198


In [37]:
# 查看无法获取正文的href链接，分析原因
speeches.loc[speeches["content_length"] == 0, "href"].tolist()

['https://www.bostonfed.org/news-and-events/speeches/municipal-strategies-for-financial-empowerment.aspx',
 'https://www.bostonfed.org/news-and-events/speeches/some-observations-on-the-international-implications-of-asset-bubbles.aspx']

波士顿：Boston无法获取到content的原因是因为的确没有提供，或者讲话数据以pdf链接的形式提供。

In [38]:
# 正文缺失的个数
missing_number = sum(speeches["content_length"] == 0)
print(f"正文缺失的个数：{missing_number}")
# 正文长度统计
content_length_desc = speeches["content_length"].describe().to_dict()
content_length_desc["missing"] = missing_number
content_length_desc

正文缺失的个数：2


{'count': 215.0,
 'mean': 7931.013953488372,
 'std': 8450.459532646051,
 'min': 0.0,
 '25%': 2151.5,
 '50%': 3018.0,
 '75%': 15945.5,
 'max': 28701.0,
 'missing': 2}

In [ ]:
# speeches.groupby(["year"]).apply(lambda gpdf: gpdf[['content_length']].describe())

# 美联储讲话数据EDA?

@AUTHOR: WBZHANG / 01208663

@DATE: Oct. 31, 2024

## 2. MM2鹰鸽指数解析

In [40]:
import pandas as pd
import numpy as np

In [71]:
lp_point = [70, 95] 
rb_point = [660, 287]

In [ ]:
import re

# SVG代码字符串
svg_code = """
<g class="highcharts-axis-labels highcharts-xaxis-labels" data-z-index="7">
    <text x="119.60535362875" style="color: rgb(102, 102, 102); cursor: default; font-size: 11px; fill: rgb(102, 102, 102);" text-anchor="middle" transform="translate(0,0)" y="313" opacity="1">2008</text>
    <text x="182.43284254451" style="color: rgb(102, 102, 102); cursor: default; font-size: 11px; fill: rgb(102, 102, 102);" text-anchor="middle" transform="translate(0,0)" y="313" opacity="1">2010</text>
    <text x="245.17438414302" style="color: rgb(102, 102, 102); cursor: default; font-size: 11px; fill: rgb(102, 102, 102);" text-anchor="middle" transform="translate(0,0)" y="313" opacity="1">2012</text>
    <text x="308.00187305878" style="color: rgb(102, 102, 102); cursor: default; font-size: 11px; fill: rgb(102, 102, 102);" text-anchor="middle" transform="translate(0,0)" y="313" opacity="1">2014</text>
    <text x="370.74341465728" style="color: rgb(102, 102, 102); cursor: default; font-size: 11px; fill: rgb(102, 102, 102);" text-anchor="middle" transform="translate(0,0)" y="313" opacity="1">2016</text>
    <text x="433.57090357304" style="color: rgb(102, 102, 102); cursor: default; font-size: 11px; fill: rgb(102, 102, 102);" text-anchor="middle" transform="translate(0,0)" y="313" opacity="1">2018</text>
    <text x="496.31244517154" style="color: rgb(102, 102, 102); cursor: default; font-size: 11px; fill: rgb(102, 102, 102);" text-anchor="middle" transform="translate(0,0)" y="313" opacity="1">2020</text>
    <text x="559.1399340873" style="color: rgb(102, 102, 102); cursor: default; font-size: 11px; fill: rgb(102, 102, 102);" text-anchor="middle" transform="translate(0,0)" y="313" opacity="1">2022</text>
    <text x="621.8814756858" style="color: rgb(102, 102, 102); cursor: default; font-size: 11px; fill: rgb(102, 102, 102);" text-anchor="middle" transform="translate(0,0)" y="313" opacity="1">2024</text>
</g>
"""

# 使用正则表达式提取年份
x_axis_labels = re.findall(r"<text[^>]*>([\d]{4})</text>", svg_code)
# 使用正则表达式提取x坐标
x_axis = re.findall(r"<text x=\"(.*)\" style=", svg_code)

# 打印提取的X轴标签
print(x_axis_labels)
print(x_axis)

xaxis_labels = {f"{label}-01-01": float(x) for label, x in zip(x_axis_labels, x_axis)}
xaxis_labels

['2008', '2010', '2012', '2014', '2016', '2018', '2020', '2022', '2024']
['119.60535362875', '182.43284254451', '245.17438414302', '308.00187305878', '370.74341465728', '433.57090357304', '496.31244517154', '559.1399340873', '621.8814756858']


{'2008-01-01': 119.60535362875,
 '2010-01-01': 182.43284254451,
 '2012-01-01': 245.17438414302,
 '2014-01-01': 308.00187305878,
 '2016-01-01': 370.74341465728,
 '2018-01-01': 433.57090357304,
 '2020-01-01': 496.31244517154,
 '2022-01-01': 559.1399340873,
 '2024-01-01': 621.8814756858}

In [85]:
def minmax(sr):
    min_v, max_v = sr.min(), sr.max()
    return  (sr - min_v)*100 / (max_v - min_v)

# 读取预处理的highcharts数据点
mm2 = pd.read_excel("../data/MM2/MM2.xlsx")
mm2.y = -mm2.y
mm2.y = minmax(mm2.y)
mm2

,x,y
0,5.686275,94.3172
1,9.382009,95.5970
2,12.390165,97.7844
3,16.515636,99.2530
4,20.813002,99.5057
...,...,...
144,558.671314,58.3669
145,562.281101,60.5333
146,565.890888,59.2741
147,570.102307,58.8235


### 2.1 日期映射

#### 格里高利历

In [193]:
gregorian = []
for date in xaxis_labels:
    # 日期
    ordinal = pd.to_datetime(date).toordinal()
    # 赋值
    gregorian.append({"ordinal": ordinal, "x": xaxis_labels[date]})
gregorian = pd.DataFrame.from_records(gregorian)

# 计算斜率 -> 每刻度y对应的julian数值
slope = (gregorian["ordinal"].max() - gregorian["ordinal"].min()) / (
    gregorian["x"].max() - gregorian["x"].min()
)
print(f"Slope of gregorian date number of per x: {slope}")

gregorian["slope"] = (gregorian["ordinal"] - gregorian["ordinal"].shift(1)) / (
    gregorian["x"] - gregorian["x"].shift(1)
)
gregorian["slope"] = gregorian["slope"].bfill()

def x2date_by_gregorian(x: float, xshift: float = 70):
    """根据x坐标和格里高利历史的斜率来推算日期

    Args:
        x (float): _description_

    Returns:
        _type_: _description_
    """
    # 计算系数
    number = gregorian.loc[0, "ordinal"] + slope * (x - gregorian.loc[0, "x"] + xshift)
    # datetime_date = pd.to_datetime(number, unit="s").date()
    datetime_date = pd.Timestamp.fromordinal(int(number.round())).date()
    return datetime_date

print(x2date_by_gregorian(mm2.x[0]))

mm2["date"] = mm2["x"].transform(x2date_by_gregorian)
# 日期
mm2["MM2_Hawkish-Dovish_Index"] = mm2["y"]
mm2[["date", "MM2_Hawkish-Dovish_Index"]]

Slope of gregorian date number of per x: 11.635034482758513
2006-08-08


,date,MM2_Hawkish-Dovish_Index
0,2006-08-08,94.3172
1,2006-09-20,95.5970
2,2006-10-25,97.7844
3,2006-12-12,99.2530
4,2007-01-31,99.5057
...,...,...
144,2024-03-20,58.3669
145,2024-05-01,60.5333
146,2024-06-12,59.2741
147,2024-07-31,58.8235


### 2.2 MM2鹰鸽指数绘制

In [169]:
import plotly.graph_objects as go

# 数据点，从SVG路径中提取
x = mm2.date
y = mm2.y

# 创建一个图形对象
fig = go.Figure()

# 添加一条折线图
fig.add_trace(
    go.Scatter(
        x=x,
        y=y,
        mode="lines+markers",
        name="美国-MM AI 美联储声明稿鹰鸽指数",
        line={"width": 0.8},
    )  # , marker={"size":4}
)

# 设置图形的标题和坐标轴标签
fig.update_layout(
    title="美国-MM AI美联储声明稿鹰鸽指数",
    xaxis_title="日期",
    yaxis_title="指数",
    xaxis=dict(
        rangeselector=dict(
            buttons=list(
                [
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(count=3, label="3y", step="year", stepmode="backward"),
                    dict(step="all"),
                ]
            )
        )
    ),
    yaxis=dict(rangemode="tozero"),
)

# 显示图形
fig.show()
fig.write_html("../data/MM2/美国-MM AI美联储声明稿鹰鸽指数.html")

In [240]:
mm2['MM2_Hawkish-Dovish_Index'] = mm2['y']
mm2[["date", "MM2_Hawkish-Dovish_Index"]]
mm2[['date', 'MM2_Hawkish-Dovish_Index']].to_csv("../data/MM2_Hawkish-Dovish_Index.csv", index=False)

## 3. MM2 声明稿乐观指数

### 3.1 数据转换

In [229]:
# x轴坐标
xlabels_coords = """
<g class="highcharts-axis-labels highcharts-xaxis-labels" data-z-index="7"><text x="236.516916803" style="color: rgb(102, 102, 102); cursor: default; font-size: 11px; fill: rgb(102, 102, 102);" text-anchor="middle" transform="translate(0,0)" y="425" opacity="1">2010</text><text x="701.38343172819" style="color: rgb(102, 102, 102); cursor: default; font-size: 11px; fill: rgb(102, 102, 102);" text-anchor="middle" transform="translate(0,0)" y="425" opacity="1">2020</text><text x="143.46723925362" style="color: rgb(102, 102, 102); cursor: default; font-size: 11px; fill: rgb(102, 102, 102);" text-anchor="middle" transform="translate(0,0)" y="425" opacity="1">2008</text><text x="329.43930341181" style="color: rgb(102, 102, 102); cursor: default; font-size: 11px; fill: rgb(102, 102, 102);" text-anchor="middle" transform="translate(0,0)" y="425" opacity="1">2012</text><text x="422.48898096119" style="color: rgb(102, 102, 102); cursor: default; font-size: 11px; fill: rgb(102, 102, 102);" text-anchor="middle" transform="translate(0,0)" y="425" opacity="1">2014</text><text x="515.41136757" style="color: rgb(102, 102, 102); cursor: default; font-size: 11px; fill: rgb(102, 102, 102);" text-anchor="middle" transform="translate(0,0)" y="425" opacity="1">2016</text><text x="608.46104511938" style="color: rgb(102, 102, 102); cursor: default; font-size: 11px; fill: rgb(102, 102, 102);" text-anchor="middle" transform="translate(0,0)" y="425" opacity="1">2018</text><text x="794.43310927757" style="color: rgb(102, 102, 102); cursor: default; font-size: 11px; fill: rgb(102, 102, 102);" text-anchor="middle" transform="translate(0,0)" y="425" opacity="1">2022</text><text x="887.35549588638" style="color: rgb(102, 102, 102); cursor: default; font-size: 11px; fill: rgb(102, 102, 102);" text-anchor="middle" transform="translate(0,0)" y="425" opacity="1">2024</text></g>
""".replace(">", ">\n")
# # 使用正则表达式提取年份
xaxis_dates = re.findall(r">\n([\d]{4})</text>", xlabels_coords)
print(f"xaxis_dates: {xaxis_dates}")
# # 使用正则表达式提取x坐标
xaxis_cords = re.findall(r"<text x=\"(.*)\" style=", xlabels_coords)
print(f"xaxis_cords: {xaxis_cords}")

xaxis_dates: ['2010', '2020', '2008', '2012', '2014', '2016', '2018', '2022', '2024']
xaxis_cords: ['236.516916803', '701.38343172819', '143.46723925362', '329.43930341181', '422.48898096119', '515.41136757', '608.46104511938', '794.43310927757', '887.35549588638']


In [237]:
gregorian = []
for date, cord in zip(xaxis_dates, xaxis_cords):
    # 日期
    ordinal = pd.to_datetime(f"{date}-01-01").toordinal()
    # 赋值
    gregorian.append({"date": f"{date}-01-01", "ordinal": ordinal, "x": float(cord)})
gregorian = pd.DataFrame.from_records(gregorian).sort_values("ordinal")

# 计算斜率 -> 每刻度y对应的julian数值
slope = (gregorian["ordinal"].max() - gregorian["ordinal"].min()) / (
    gregorian["x"].max() - gregorian["x"].min()
)
print(f"Slope of gregorian date number of per x: {slope}")

gregorian["slope"] = (gregorian["ordinal"] - gregorian["ordinal"].shift(1)) / (
    gregorian["x"] - gregorian["x"].shift(1)
)
gregorian["slope"] = gregorian["slope"].bfill()
gregorian

Slope of gregorian date number of per x: 7.856018626309683


,date,ordinal,x,slope
2,2008-01-01,733042,143.467239,7.856019
0,2010-01-01,733773,236.516917,7.856019
3,2012-01-01,734503,329.439303,7.856019
4,2014-01-01,735234,422.488981,7.856019
5,2016-01-01,735964,515.411368,7.856019
6,2018-01-01,736695,608.461045,7.856019
1,2020-01-01,737425,701.383432,7.856019
7,2022-01-01,738156,794.433109,7.856019
8,2024-01-01,738886,887.355496,7.856019


In [238]:
def x2date_by_gregorian(x: float, xshift: float = 70):
    """根据x坐标和格里高利历史的斜率来推算日期

    Args:
        x (float): _description_

    Returns:
        _type_: _description_
    """
    # 计算系数
    number = gregorian.loc[0, "ordinal"] + slope * (x - gregorian.loc[0, "x"] + xshift)
    # datetime_date = pd.to_datetime(number, unit="s").date()
    datetime_date = pd.Timestamp.fromordinal(int(number.round())).date()
    return datetime_date

In [239]:
points_str = """
M 8.42157 40.3162 L 13.8951 45.9189 L 18.3503 45.0507 L 24.4602 55.7718 L 30.8248 52.3974 L 37.062 48.8203 L 43.2993 40.3511 L 48.3909 31.9148 L 54.7555 11.3097 L 60.1017 56.2336 L 65.5752 84.4661 L 70.7941 131.992 L 77.1587 201.946 L 83.2686 267.353 L 88.7421 275.744 L 95.8704 257.213 L 101.089 211.575 L 106.436 164.229 L 111.909 179.653 L 118.019 182.814 L 123.493 213.259 L 129.73 266.156 L 135.076 304 L 142.204 287.795 L 148.442 269.651 L 153.788 266.21 L 159.134 252.933 L 164.48 240.102 L 169.826 231.116 L 175.936 231.682 L 181.41 227.223 L 188.538 220.025 L 194.648 218.581 L 199.994 213.729 L 205.468 211.616 L 210.687 210.581 L 216.16 198.184 L 222.27 165.366 L 227.744 135.169 L 234.872 130.988 L 240.982 142.364 L 246.456 142.895 L 251.802 155.486 L 257.021 167.675 L 262.494 168.19 L 268.604 156.608 L 274.078 160.728 L 281.206 154.313 L 286.552 172.389 L 292.026 163.622 L 297.245 160.326 L 303.482 170.396 L 309.719 184.597 L 315.956 168.857 L 321.303 171.432 L 327.54 172.529 L 332.886 165.205 L 339.123 156.691 L 344.47 156.306 L 350.707 158.363 L 356.053 150.887 L 362.29 155.243 L 367.637 146.464 L 373.874 123.704 L 379.22 101.992 L 385.457 98.0272 L 390.804 83.8858 L 397.041 84.2001 L 402.387 92.5306 L 408.624 92.6017 L 413.971 92.585 L 420.208 77.0825 L 425.554 62.7468 L 431.919 58.731 L 437.137 57.1657 L 443.375 51.8904 L 448.721 65.3488 L 454.958 80.4503 L 460.304 83.3139 L 466.542 88.7349 L 471.888 86.9784 L 479.016 77.8343 L 484.362 74.7776 L 489.709 79.6757 L 495.946 80.7746 L 501.292 82.7564 L 507.529 83.0571 L 512.876 87.2164 L 518.222 80.1645 L 525.35 76.9071 L 530.696 81.9794 L 536.043 95.0888 L 542.28 95.5755 L 548.517 98.7635 L 553.863 101.705 L 559.209 83.8396 L 565.447 64.2081 L 572.575 47.8952 L 578.049 36.9977 L 583.267 25.9975 L 588.614 34.2754 L 594.851 47.721 L 600.197 54.796 L 606.434 63.6892 L 611.781 80.8196 L 618.018 91.0571 L 620.946 81.6787 L 623.364 95.3952 L 628.71 99.3895 L 634.948 103.148 L 639.275 133.139 L 640.803 156.295 L 641.821 174.906 L 646.531 160.564 L 651.877 164.248 L 658.115 160.402 L 664.352 119.258 L 670.716 92.2309 L 675.935 79.2765 L 681.281 72.3398 L 687.519 66.1756 L 692.865 55.5584 L 699.102 38.6366 L 704.448 17.7688 L 711.577 2.80653 L 716.923 0 L 722.269 8.46184 L 727.615 15.7657 L 733.853 42.7397 L 740.09 72.6682 L 745.436 87.1416 L 750.782 89.179 L 757.911 87.8776 L 763.257 79.137 L 768.603 64.7961 L 774.84 62.5304 L 781.078 62.2066 L 786.424 69.2637 L 791.77 66.272 L 797.116 60.8371 L 804.245 53.1489 L 809.591 51.9998 L 814.937 56.9392 L 821.174 56.706 L 827.411 57.1791 L 832.758 56.8322 L 838.104 50.8622 L 844.341 31.716 L 850.578 40.7555
"""
points_str = points_str.replace(" L ", "\n").replace("M ", "").strip()
points = [tuple(map(float, p.split(" "))) for p in points_str.split("\n")]
points = pd.DataFrame(points, columns=["date_cord", "index"])
points["index"] = minmax(-1 * points["index"])
# 将坐标映射为日期
points['date'] = points['date_cord'].transform(x2date_by_gregorian, xshift=70)
points

,date_cord,index,date
0,8.42157,86.738092,2006-08-08
1,13.89510,84.895099,2006-09-20
2,18.35030,85.180691,2006-10-25
3,24.46020,81.654013,2006-12-12
4,30.82480,82.764013,2007-01-31
...,...,...,...
144,827.41100,81.191086,2024-03-20
145,832.75800,81.305197,2024-05-01
146,838.10400,83.269013,2024-06-12
147,844.34100,89.567105,2024-07-31


In [245]:
import plotly.graph_objects as go

# 创建一个图形对象
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=mm2['date'],
        y=mm2['y'],
        mode="lines+markers",
        name="美国-MM AI 美联储声明稿鹰鸽指数",
        line={"width": 0.8},
    )  # , marker={"size":4}
)

fig.add_trace(
    go.Scatter(
        x=points["date"],
        y=points["index"],
        mode="lines+markers",
        name="美国-MM AI 美联储声明稿乐观指数",
        line={"width": 0.8},
    )  # , marker={"size":4}
)

# 设置图形的标题和坐标轴标签
fig.update_layout(
    title="美国-MM AI美联储声明稿乐观指数",
    xaxis_title="日期",
    yaxis_title="指数",
    xaxis=dict(
        rangeselector=dict(
            buttons=list(
                [
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(count=3, label="3y", step="year", stepmode="backward"),
                    dict(step="all"),
                ]
            )
        )
    ),
    yaxis=dict(rangemode="tozero"),
)

# 显示图形
fig.show()
fig.write_html("../data/MM2/美国-MM AI美联储声明稿乐观指数.html")

points[['date', 'index']].to_csv("../data/MM2/美国-MM AI美联储声明稿乐观指数.csv", index=False)
